In [ ]:
\from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Paths
root_dir = "/content/drive/MyDrive/Frames_CNN"
output_dir = "/content/drive/MyDrive/Frames_seqential_split"
labels_path = "/content/drive/MyDrive/Infant_data.csv"

# Load labels
labels_df = pd.read_csv(labels_path)

# Collect all infants and their classes
infant_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

infant_ids = []
classes = []
for folder in infant_folders:
    if "motion" in folder:
        infant_num = folder.split("motion")[1].split("_")[0]
        infant_id = f"baby{infant_num}"

        label_row = labels_df[labels_df['infant_id'] == infant_id]
        if not label_row.empty:
            infant_ids.append(folder)
            classes.append(label_row['class'].values[0])

# Split infants → train/test
train_ids, test_ids, y_train, y_test = train_test_split(
    infant_ids, classes, test_size=0.2, random_state=42, stratify=classes
)

# Create output dirs
for split in ["train", "test"]:
    for cls in ["0", "1"]:
        os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

# Copy files infant-wise
def copy_infants(infant_list, split):
    for folder in infant_list:
        infant_num = folder.split("motion")[1].split("_")[0]
        infant_id = f"baby{infant_num}"

        label = labels_df[labels_df['infant_id'] == infant_id]['class'].values[0]
        src = os.path.join(root_dir, folder)
        dst = os.path.join(output_dir, split, str(label), folder)
        shutil.copytree(src, dst, dirs_exist_ok=True)

copy_infants(train_ids, "train")
copy_infants(test_ids, "test")

print("✅ Dataset split by infants completed!")


✅ Dataset split by infants completed!


In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class SequenceDataset(Dataset):
    def __init__(self, root_dir, transform=None, seq_len=30):
        """
        root_dir: path to train/0, train/1 etc.
        transform: transformations for frames
        seq_len: fixed number of frames per sequence
        """
        self.root_dir = root_dir
        self.transform = transform
        self.seq_len = seq_len
        self.samples = []

        # Go through each class (0, 1)
        for label in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label)
            if not os.path.isdir(class_dir):
                continue

            # Each clip folder is one sample
            for clip in os.listdir(class_dir):
                clip_path = os.path.join(class_dir, clip)
                if os.path.isdir(clip_path):
                    self.samples.append((clip_path, int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        clip_path, label = self.samples[idx]

        # Get all frames in order
        frames = sorted(os.listdir(clip_path))
        frames_data = []

        for frame_name in frames[:self.seq_len]:  # limit to seq_len frames
            frame_path = os.path.join(clip_path, frame_name)
            img = cv2.imread(frame_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if self.transform:
                img = self.transform(img)

            frames_data.append(img)

        # If less frames than seq_len → pad
        while len(frames_data) < self.seq_len:
            frames_data.append(torch.zeros_like(frames_data[0]))

        # Shape: (seq_len, C, H, W)
        frames_tensor = torch.stack(frames_data)

        return frames_tensor, label


In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128,128)),
    transforms.ToTensor()
])


In [ ]:
from torch.utils.data import DataLoader

train_dir = "/content/drive/MyDrive/Frames_sequential_split/train"
test_dir  = "/content/drive/MyDrive/Frames_sequential_split/test"

train_dataset = SequenceDataset(train_dir, transform=transform, seq_len=30)
test_dataset  = SequenceDataset(test_dir, transform=transform, seq_len=30)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f"Train clips: {len(train_dataset)}, Test clips: {len(test_dataset)}")


Train clips: 61, Test clips: 16


In [ ]:
sample, label = train_dataset[0]
print("Sample shape:", sample.shape)  # (seq_len, C, H, W)
print("Label:", label)


Sample shape: torch.Size([30, 3, 128, 128])
Label: 0


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CNN_LSTM(nn.Module):
    def __init__(self, cnn_embed_dim=512, lstm_hidden=128, lstm_layers=1, num_classes=1):
        super(CNN_LSTM, self).__init__()

        # Step 1: Pretrained CNN (feature extractor)
        self.cnn = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT).features
        self.cnn_pool = nn.AdaptiveAvgPool2d((1, 1))  # output shape = (batch, 1280, 1, 1)
        self.cnn_fc = nn.Linear(1280, cnn_embed_dim)  # reduce to 512-dim feature

        # Step 2: LSTM for temporal modeling
        self.lstm = nn.LSTM(input_size=cnn_embed_dim,
                            hidden_size=lstm_hidden,
                            num_layers=lstm_layers,
                            batch_first=True)

        # Step 3: Fully connected for classification
        self.fc1 = nn.Linear(lstm_hidden, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.sigmoid = nn.Sigmoid()  # binary classification

    def forward(self, x):
        # x: (batch, seq_len, 3, 128, 128)
        batch_size, seq_len, C, H, W = x.size()

        # Flatten batch & seq to feed frames into CNN
        x = x.view(batch_size * seq_len, C, H, W)
        with torch.no_grad():  # freeze CNN weights if you want
            cnn_features = self.cnn(x)             # (batch*seq_len, 1280, H', W')
            cnn_features = self.cnn_pool(cnn_features)  # (batch*seq_len, 1280, 1, 1)
            cnn_features = cnn_features.view(batch_size*seq_len, -1)  # (batch*seq_len, 1280)
            cnn_features = self.cnn_fc(cnn_features)  # (batch*seq_len, 512)

        # Reshape back to sequence
        cnn_features = cnn_features.view(batch_size, seq_len, -1)  # (batch, seq_len, 512)

        # LSTM
        lstm_out, (h_n, c_n) = self.lstm(cnn_features)  # lstm_out: (batch, seq_len, hidden)
        last_output = lstm_out[:, -1, :]                 # take last time-step output

        # Classification
        x = self.fc1(last_output)
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x


In [ ]:
#Testing the model forward pass
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM().to(device)

# Example: batch of 4 sequences
for batch_x, batch_y in train_loader:
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device).float().unsqueeze(1)
    out = model(batch_x)
    print("Output shape:", out.shape)  # should be (batch_size, 1)
    break


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 91.9MB/s]


Output shape: torch.Size([4, 1])


In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

for epoch in range(num_epochs):
    # ===== TRAINING =====
    model.train()
    running_loss = 0
    running_corrects = 0
    total_samples = 0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        preds = (outputs > 0.5).float()
        running_corrects += (preds == y_batch).sum().item()
        total_samples += X_batch.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc  = running_corrects / total_samples

    # ===== VALIDATION =====
    model.eval()
    val_loss = 0
    val_corrects = 0
    val_samples = 0

    with torch.no_grad():
        for X_val, y_val in test_loader:
            X_val = X_val.to(device)
            y_val = y_val.to(device).float().unsqueeze(1)

            outputs_val = model(X_val)
            loss_val = criterion(outputs_val, y_val)

            val_loss += loss_val.item() * X_val.size(0)
            preds_val = (outputs_val > 0.5).float()
            val_corrects += (preds_val == y_val).sum().item()
            val_samples += X_val.size(0)

    val_epoch_loss = val_loss / val_samples
    val_epoch_acc  = val_corrects / val_samples

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


Epoch 1/10 | Train Loss: 0.6926, Train Acc: 0.5246 | Val Loss: 0.6922, Val Acc: 0.4375
Epoch 2/10 | Train Loss: 0.6906, Train Acc: 0.6230 | Val Loss: 0.6898, Val Acc: 0.5625
Epoch 3/10 | Train Loss: 0.6858, Train Acc: 0.6066 | Val Loss: 0.6862, Val Acc: 0.6250
Epoch 4/10 | Train Loss: 0.6796, Train Acc: 0.6557 | Val Loss: 0.6820, Val Acc: 0.5625
Epoch 5/10 | Train Loss: 0.6734, Train Acc: 0.6066 | Val Loss: 0.6770, Val Acc: 0.5625
Epoch 6/10 | Train Loss: 0.6677, Train Acc: 0.6066 | Val Loss: 0.6721, Val Acc: 0.5625
Epoch 7/10 | Train Loss: 0.6617, Train Acc: 0.6066 | Val Loss: 0.6667, Val Acc: 0.6250
Epoch 8/10 | Train Loss: 0.6553, Train Acc: 0.5902 | Val Loss: 0.6534, Val Acc: 0.6250
Epoch 9/10 | Train Loss: 0.6392, Train Acc: 0.6230 | Val Loss: 0.6374, Val Acc: 0.6250
Epoch 10/10 | Train Loss: 0.5999, Train Acc: 0.6885 | Val Loss: 0.6004, Val Acc: 0.7500


In [ ]:
#Full Fine-Tuned CNN + LSTM Code with Data Augmentation

In [ ]:
# ============================================================
# 1️⃣  Imports & Setup
# ============================================================
import os
import random
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", device)

# ============================================================
# 2️⃣  Data Augmentation & Normalization
# ============================================================
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ============================================================
# 3️⃣  Sequence Dataset (CNN-LSTM Ready)
# ============================================================
class SequenceDataset(Dataset):
    def __init__(self, root_dir, transform=None, seq_len=20):
        self.root_dir = root_dir
        self.transform = transform
        self.seq_len = seq_len
        self.samples = []

        for label in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label)
            if not os.path.isdir(class_dir):
                continue
            for clip in os.listdir(class_dir):
                clip_path = os.path.join(class_dir, clip)
                if os.path.isdir(clip_path):
                    self.samples.append((clip_path, int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        clip_path, label = self.samples[idx]
        frames = sorted(os.listdir(clip_path))
        frames_data = []

        # Random start for better generalization
        start_idx = random.randint(0, max(0, len(frames) - self.seq_len))
        selected_frames = frames[start_idx : start_idx + self.seq_len]

        for frame_name in selected_frames:
            frame_path = os.path.join(clip_path, frame_name)
            img = cv2.imread(frame_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.transform:
                img = self.transform(img)
            frames_data.append(img)

        # Pad if fewer frames
        while len(frames_data) < self.seq_len:
            frames_data.append(torch.zeros_like(frames_data[0]))

        frames_tensor = torch.stack(frames_data)  # (seq_len, C, H, W)
        return frames_tensor, label

# ============================================================
# 4️⃣  Load Dataset
# ============================================================
train_dataset = SequenceDataset(
    "/content/drive/MyDrive/Frames_sequential_split/train", transform=train_transforms, seq_len=20)
test_dataset = SequenceDataset(
    "/content/drive/MyDrive/Frames_sequential_split/test", transform=test_transforms, seq_len=20)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_dataset)} | Test samples: {len(test_dataset)}")

# ============================================================
# 5️⃣  CNN + LSTM Model with MobileNetV2 Backbone
# ============================================================
class CNN_LSTM(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=1, num_classes=2):
        super(CNN_LSTM, self).__init__()
        mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        self.cnn = mobilenet.features
        self.cnn_out_dim = 1280  # MobileNetV2 output channels

        self.lstm = nn.LSTM(input_size=self.cnn_out_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=0.3)

        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # (B, T, C, H, W)
        b, t, c, h, w = x.size()
        cnn_out = []
        for i in range(t):
            frame_feat = self.cnn(x[:, i, :, :, :])
            frame_feat = torch.mean(frame_feat, dim=[2, 3])  # Global Average Pool
            cnn_out.append(frame_feat)
        cnn_out = torch.stack(cnn_out, dim=1)  # (B, T, feat)
        lstm_out, _ = self.lstm(cnn_out)
        out = self.fc(lstm_out[:, -1, :])
        return out

# ============================================================
# 6️⃣  Initialize Model, Loss, Optimizer
# ============================================================
model = CNN_LSTM().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ============================================================
# 7️⃣  Training Loop with Checkpoint
# ============================================================
best_val_acc = 0.0
num_epochs = 25

for epoch in range(1, num_epochs + 1):
    # ---- Training ----
    model.train()
    train_loss, train_correct, total = 0, 0, 0
    for frames, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
        frames, labels = frames.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = train_correct / total

    # ---- Validation ----
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for frames, labels in val_loader:
            frames, labels = frames.to(device), labels.to(device)
            outputs = model(frames)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    scheduler.step()

    print(f"Epoch [{epoch}/{num_epochs}] "
          f"| Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_cnn_lstm_model.pth")
        print("✅ Saved best model!")

print(f"🏁 Training completed. Best Val Accuracy: {best_val_acc:.4f}")


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


✅ Device: cuda
Train samples: 61 | Test samples: 16


Epoch 1/25: 100%|██████████| 16/16 [00:31<00:00,  1.96s/it]


Epoch [1/25] | Train Loss: 0.6546, Train Acc: 0.6066 | Val Loss: 0.4318, Val Acc: 0.8750
✅ Saved best model!


Epoch 2/25: 100%|██████████| 16/16 [00:31<00:00,  1.96s/it]


...

Epoch [23/25] | Train Loss: 0.2268, Train Acc: 0.8689 | Val Loss: 0.0930, Val Acc: 1.0000


Epoch 24/25: 100%|██████████| 16/16 [00:34<00:00,  2.13s/it]


Epoch [24/25] | Train Loss: 0.2556, Train Acc: 0.9180 | Val Loss: 0.9017, Val Acc: 0.5625


Epoch 25/25: 100%|██████████| 16/16 [00:37<00:00,  2.36s/it]


Epoch [25/25] | Train Loss: 0.3015, Train Acc: 0.9016 | Val Loss: 0.6717, Val Acc: 0.6250
🏁 Training completed. Best Val Accuracy: 1.0000


In [ ]:
#Early Stopping

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# your CNN + LSTM model here (keep as is)
# model = CNN_LSTM(...)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

num_epochs = 25
patience = 3  # stop if validation acc doesn’t improve for 3 consecutive epochs
best_val_acc = 0.0
epochs_no_improve = 0
best_model_path = "best_cnn_lstm_model.pth"

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = correct / total

    # ---- Validation ----
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # ---- Early Stopping Logic ----
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Saved best model at epoch {epoch+1} (Val Acc: {val_acc:.4f})")
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement for {epochs_no_improve} epochs.")

    if epochs_no_improve >= patience:
        print(f"\n🛑 Early stopping at epoch {epoch+1}")
        break

print(f"\nTraining completed. Best validation accuracy: {best_val_acc:.4f}")


Epoch [1/25] | Train Loss: 0.2499, Train Acc: 0.9016 | Val Loss: 0.9514, Val Acc: 0.5625
✅ Saved best model at epoch 1 (Val Acc: 0.5625)
Epoch [2/25] | Train Loss: 0.1434, Train Acc: 0.9508 | Val Loss: 0.1281, Val Acc: 0.9375
✅ Saved best model at epoch 2 (Val Acc: 0.9375)
Epoch [3/25] | Train Loss: 0.4749, Train Acc: 0.8033 | Val Loss: 0.8427, Val Acc: 0.6250
⚠️ No improvement for 1 epochs.
Epoch [4/25] | Train Loss: 0.1961, Train Acc: 0.9180 | Val Loss: 1.0305, Val Acc: 0.5625
⚠️ No improvement for 2 epochs.
Epoch [5/25] | Train Loss: 0.1866, Train Acc: 0.9344 | Val Loss: 0.2421, Val Acc: 0.9375
⚠️ No improvement for 3 epochs.

🛑 Early stopping at epoch 5

Training completed. Best validation accuracy: 0.9375


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Make sure you have the same model class used during training ---
# Example: model = CNN_LSTM(...)
model = CNN_LSTM(num_classes=2)   # or the correct number of classes
model.load_state_dict(torch.load("best_cnn_lstm_model.pth", map_location=device))
model.to(device)
model.eval()

print("✅ Best model loaded for prediction.")


✅ Best model loaded for prediction.


In [ ]:
from PIL import Image
import os
import torch

# Use same transform as test_transforms used in training
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load frames
def load_sequence_frames(folder_path, seq_len=20):
    frames = sorted(os.listdir(folder_path))[:seq_len]
    imgs = []
    for f in frames:
        img_path = os.path.join(folder_path, f)
        img = Image.open(img_path).convert("RGB")
        img = test_transforms(img)
        imgs.append(img)
    # Shape: (seq_len, 3, 224, 224)
    sequence = torch.stack(imgs)
    # Add batch dimension: (1, seq_len, 3, 224, 224)
    return sequence.unsqueeze(0)

# Example
sequence = load_sequence_frames("/content/drive/MyDrive/Frames_seqential_split/test/0/skeleton_slow_resized_baby_motion10_25fps_clip1", seq_len=20).to(device)


In [ ]:
model.eval()
with torch.no_grad():
    output = model(sequence)
    _, predicted = torch.max(output, 1)
    print(f"Predicted class: {predicted.item()}")


Predicted class: 0


In [ ]:
label_map = {0: "Normal", 1: "Abnormal"}
print("Predicted:", label_map[predicted.item()])


Predicted: Normal


In [ ]:
from PIL import Image
import os
import torch

# Use same transform as test_transforms used in training
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load frames
def load_sequence_frames(folder_path, seq_len=20):
    frames = sorted(os.listdir(folder_path))[:seq_len]
    imgs = []
    for f in frames:
        img_path = os.path.join(folder_path, f)
        img = Image.open(img_path).convert("RGB")
        img = test_transforms(img)
        imgs.append(img)
    # Shape: (seq_len, 3, 224, 224)
    sequence = torch.stack(imgs)
    # Add batch dimension: (1, seq_len, 3, 224, 224)
    return sequence.unsqueeze(0)

# Example
sequence = load_sequence_frames("/content/drive/MyDrive/Frames_seqential_split/test/1/skeleton_slow_resized_baby_motion8_25fps_clip1", seq_len=20).to(device)


In [ ]:
model.eval()
with torch.no_grad():
    output = model(sequence)
    _, predicted = torch.max(output, 1)
    print(f"Predicted class: {predicted.item()}")


Predicted class: 1


In [ ]:
label_map = {0: "Normal", 1: "Abnormal"}
print("Predicted:", label_map[predicted.item()])


Predicted: Abnormal
